In [1]:
from sympy import *
import numpy as np
import sympy as sp

In [2]:
# \Gamma^{\mu}_{\nu \beta} = g^{\mu \nu} = \Gamma_{\nu \alpha \beta} 
# where \Gamma_{\nu \alpha \beta} = \frac{1}{2} [ {\d_{x^{\beta}}g_{\mu \alpha}} + {\d_{x^{\alpha}}g_{\mu \beta}} 
# - {\d_{x^{\mu}}g_{\alpha \beta}} ]


In [10]:
# Class to calculate christoffel symbol for Schwarzschild metric
# ds^2 = -(1-2M/r) dt^2 + 1/(1-2M/r) dr^2 + r^2 d\theta^2 + r^2 sin(\theta)^2 d\phi^2

class Christoffel_Symbol:
    #define the symbols
    global M, r, theta, phi 
    #global r
    #global theta
    #global phi
    M,t,r,theta,phi = symbols("M t r theta phi",real=True)
    #define the coordinate system (Here used spherical polar r,theta,phi)
    xi = [t,r,theta,phi]
 
    #define the Schwarzschild metric
    def Schwarzschild_Metric(self,i,j):
        g00 = -(1-2*M/r)
        g11 = 1/(1-2*M/r)
        g22 = r**2
        g33 = r**2*sin(theta)**2
        return ([[g00,0,0,0],[0,g11,0,0],[0,0,g22,0],[0,0,0,g33]])[i][j]
    
    #define derivative of the metric
    def deriv_m(self,i,j,k):
        return sp.diff(self.Schwarzschild_Metric(i,j),self.xi[k])
    
    #define inverse of the metric
    def Inv_m(self,i,j):
        K = Matrix([[self.Schwarzschild_Metric(0,0),self.Schwarzschild_Metric(0,1),self.Schwarzschild_Metric(0,2),self.Schwarzschild_Metric(0,3)],
                [self.Schwarzschild_Metric(1,0),self.Schwarzschild_Metric(1,1),self.Schwarzschild_Metric(1,2),self.Schwarzschild_Metric(1,3)],
                [self.Schwarzschild_Metric(2,0),self.Schwarzschild_Metric(2,1),self.Schwarzschild_Metric(2,2),self.Schwarzschild_Metric(2,3)],
                [self.Schwarzschild_Metric(3,0),self.Schwarzschild_Metric(3,1),self.Schwarzschild_Metric(3,2),self.Schwarzschild_Metric(3,3)]])
        return K.inv(method="LU")[i,j]
    
    #define christoffel symbol
    def Gamma(self,i,j,k):
        GammaSymb = 0
        for l in range(4):
            GammaSymb+=0.5*self.Inv_m(i,l)*(self.deriv_m(k,l,j)+self.deriv_m(l,j,k)-self.deriv_m(j,k,l))
        return simplify(GammaSymb)

       

In [16]:
Christoffel_Symbol().Gamma(3,2,3)

1.0/tan(theta)

In [17]:
# Class to calculate christoffel symbol for Freedmann-Lamitre-Robertson-Walker metric
# ds^2 = - dt^2 + a^2(t) [ 1/(1-K*r**2) dr^2 + r^2 d\theta^2 + r^2 sin(\theta)^2 d\phi^2 ]

class Christoffel_Symbol_FLRW:
    global K,t,r,theta,phi
    #define the symbols
    K,t,r,theta,phi = symbols("K t r theta phi",real=True)
    
    #define the coordinate system (Here used spherical polar r,theta,phi)
    xi = [t,r,theta,phi]
    a = sp.Function('a')(t)
    adot = a.diff(t)
    
    #define the metric
    def FLRW_metric(self,i,j):
        g00 = -1
        g11 = self.a**2* (1/(1-K*r**2))
        g22 = self.a**2 *r**2
        g33 = self.a**2* r**2*sin(theta)**2
        return np.array([[g00,0,0,0],[0,g11,0,0],[0,0,g22,0],[0,0,0,g33]])[i][j]
    
    #define derivative of the metric
    def deriv_m(self,i,j,k):
        return sp.diff(self.FLRW_metric(i,j),self.xi[k])
    
    #define inverse of metric
    def Inv_m(self,i,j):
        K = Matrix([[self.FLRW_metric(0,0),self.FLRW_metric(0,1),self.FLRW_metric(0,2),self.FLRW_metric(0,3)],
                [self.FLRW_metric(1,0),self.FLRW_metric(1,1),self.FLRW_metric(1,2),self.FLRW_metric(1,3)],
                [self.FLRW_metric(2,0),self.FLRW_metric(2,1),self.FLRW_metric(2,2),self.FLRW_metric(2,3)],
                [self.FLRW_metric(3,0),self.FLRW_metric(3,1),self.FLRW_metric(3,2),self.FLRW_metric(3,3)]])
        return K.inv(method="LU")[i,j]
    
    def Gamma(self,i,j,k):
        GammaSymb = 0
        for l in range(4):
            GammaSymb+=0.5*self.Inv_m(i,l)*(self.deriv_m(k,l,j)+self.deriv_m(l,j,k)-self.deriv_m(j,k,l))
        return simplify(GammaSymb)
    

In [19]:
Christoffel_Symbol_FLRW().Gamma(2,1,2)

1.0/r

In [20]:
# Class to calculate christoffel symbol for Perturbation Universe metric
# ds^2 = - a^2(t) [ 1+(2*Psi(x,y,z,t)) dt^2 +  1+(2*Psi(x,y,z,t)) dr^2 + r^2 d\theta^2 + r^2 sin(\theta)^2 d\phi^2 ]

class Christoffel_Symbol_Pertb_Univ:
    global t,x,y,z
    #define the symbols
    t,x,y,z = symbols("t x y z",real=True)
    
    #define the coordinate system (Here used spherical polar r,theta,phi)
    xi = [t,x,y,t]
    a = sp.Function('a')(t)
    Phi = sp.Function('Phi')(x,y,z,t)
    Psi = sp.Function('Psi')(x,y,z,t)
    
    #define the metric
    def Pertb_metric(self,i,j):
        g00 = self.a**2 * (1+(2*self.Phi))
        g11 = -self.a**2 * (1-(2*self.Psi))
        g22 = -self.a**2 * (1-(2*self.Psi))
        g33 = -self.a**2 * (1-(2*self.Psi))
        return np.array([[g00,0,0,0],[0,g11,0,0],[0,0,g22,0],[0,0,0,g33]])[i][j]
    
    #define derivative of the metric
    def deriv_m(self,i,j,k):
        return sp.diff(self.Pertb_metric(i,j),self.xi[k])
    
    #define inverse of metric
    def Inv_m(self,i,j):
        K = Matrix([[self.Pertb_metric(0,0),self.Pertb_metric(0,1),self.Pertb_metric(0,2),self.Pertb_metric(0,3)],
                [self.Pertb_metric(1,0),self.Pertb_metric(1,1),self.Pertb_metric(1,2),self.Pertb_metric(1,3)],
                [self.Pertb_metric(2,0),self.Pertb_metric(2,1),self.Pertb_metric(2,2),self.Pertb_metric(2,3)],
                [self.Pertb_metric(3,0),self.Pertb_metric(3,1),self.Pertb_metric(3,2),self.Pertb_metric(3,3)]])
        return K.inv(method="LU")[i,j]
    
    def Gamma(self,i,j,k):
        GammaSymb = 0
        for l in range(4):
            GammaSymb+=0.5*self.Inv_m(i,l)*(self.deriv_m(k,l,j)+self.deriv_m(l,j,k)-self.deriv_m(j,k,l))
        return simplify(GammaSymb)
    

In [21]:
Christoffel_Symbol_Pertb_Univ().Gamma(0,1,2)

0